In [1]:
import openmc
import openmc.mgxs as mgxs
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sp_minimized = openmc.StatePoint('./most_minimized_gen1_128/statepoint.80.h5')

In [3]:
sp_flat = openmc.StatePoint('./flat_distr_gen1_128/statepoint.80.h5')

In [4]:
sp_minimized.k_combined

1.380166946676376+/-0.0011457003311354215

In [5]:
sp_flat.k_combined

1.373604991261965+/-0.001356242264374778

In [6]:
(sp_minimized.k_combined.n - sp_flat.k_combined.n ) * 1e5

656.1955414410958

In [7]:
def get_values(sp_file):
    tally_flux = sp_file.get_tally(name='flux')
    flux = tally_flux.get_slice(scores=['flux'])
    nu_fission = tally_flux.get_slice(scores=['nu-fission'])
    fission = tally_flux.get_slice(scores=['fission'])
    flux_conv_dict = {}
    eg_names = ['eg4', 'eg3', 'eg2', 'eg1']
    egs = [(1e-6, 1.8554), (1.8554, 2.9023e1), (2.9023e1, 9.1188e3), (9.1188e3, 2.0e7)]
    flux_eg, nu, nu_fiss_eg, fiss_eg = {}, {}, {}, {}
    for x in range(4):
        print(x)
        flux_eg[x] = flux.get_slice(
            filters=[
                openmc.EnergyFilter], filter_bins=[
                (egs[x],)])
        nu_fiss_eg[x] = nu_fission.get_slice(
            filters=[
                openmc.EnergyFilter], filter_bins=[
                (egs[x],)])
        fiss_eg[x] = fission.get_slice(
            filters=[
                openmc.EnergyFilter], filter_bins=[
                (egs[x],)])
        nu[x] = sum(nu_fiss_eg[x].mean) / sum(fiss_eg[x].mean)
    return flux_eg, nu, nu_fiss_eg, fiss_eg

In [8]:
flux_eg_minimized, nu_minimized, nu_fiss_minimized, fiss_eg_minimized = get_values(sp_minimized)

0
1
2
3


In [9]:
flux_eg_flat, nu_flat, nu_fiss_flat, fiss_eg_flat = get_values(sp_flat)

0
1
2
3


In [10]:
def get_fiss_dict(fiss_eg):
    eg_names = ['eg4', 'eg3', 'eg2', 'eg1']
    fiss_dict = {}
    for x in range(4):
        fiss_dict[eg_names[x]] = fiss_eg[x].mean
        fiss_dict[eg_names[x]].shape = (100, 100)
    return fiss_dict

In [11]:
fiss_dict_minimized = get_fiss_dict(fiss_eg_minimized)

In [12]:
fiss_dict_flat = get_fiss_dict(fiss_eg_flat)

In [13]:
for i in range(1, 5):
    minimized_sum = sum(sum(fiss_dict_minimized['eg'+str(i)]))
    flat_sum = sum(sum(fiss_dict_flat['eg'+str(i)]))
    print(i, minimized_sum, flat_sum)
    print("%", (minimized_sum-flat_sum)/flat_sum*100)

1 0.0016592780059060206 0.0016264991654470482
% 2.01530017077894
2 0.008861619192618976 0.00884299737064906
% 0.2105826926028912
3 0.008545260334849076 0.008525034471897662
% 0.2372525649965102
4 0.5481347348130616 0.5446513052529579
% 0.6395705888349756


In [14]:
def total_perc(flat, minimized):
    total_flat = 0
    perc_flat = {}
    for i in range(1,5):
        total_flat += sum(sum(flat["eg"+str(i)]))
    for i in range(1,5):
        perc_flat["eg"+str(i)] = sum(sum(flat["eg"+str(i)])) / total_flat * 100
    total_minimized = 0
    perc_minimized = {}
    for i in range(1,5):
        total_minimized += sum(sum(minimized["eg"+str(i)]))
    for i in range(1,5):
        perc_minimized["eg"+str(i)] = sum(sum(minimized["eg"+str(i)])) / total_minimized * 100
    print(perc_flat)
    print(perc_minimized)

In [15]:
total_perc(fiss_dict_minimized, fiss_dict_flat)

{'eg1': 0.2925379752210553, 'eg2': 1.5623422516067667, 'eg3': 1.506566800256335, 'eg4': 96.63855297291585}
{'eg1': 0.288567582834769, 'eg2': 1.5688925211105451, 'eg3': 1.5124806968237443, 'eg4': 96.63005919923094}


In [16]:
def total_fission(fiss_dict):
    total_case = 0
    total_fiss = 0
    for i in range(1,5):
        total_case += sum(sum(fiss_dict["eg"+str(i)]))
    for i in range(1, 5):
        case_sum = sum(sum(fiss_dict['eg'+str(i)]))
        frac_case = sum(sum(fiss_dict["eg"+str(i)])) / total_case
        total_fiss += case_sum*frac_case
    return total_fiss


In [17]:
(total_fission(fiss_dict_minimized)-total_fission(fiss_dict_flat))/total_fission(fiss_dict_flat)*100

0.6480192485759656

In [18]:
total_fission(fiss_dict_minimized), total_fission(fiss_dict_flat)

(0.5299815189595772, 0.5265692488698189)